In [1]:
import mujoco
import numpy as np
import mediapy as media
np.set_printoptions(precision=3, suppress=True, linewidth=100)

In [2]:
pole1_x_pos = 0.3
pole2_x_pos = -0.4

z_coord_of_plane = -0.5
pole1_height = 0.4
pole2_height = 0.3


MJCF = f"""
<mujoco>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
    <material name="grid" texture="grid" texrepeat="1 1"
     texuniform="true" reflectance=".2"/>
  </asset>

  <worldbody>
    <light name="light" pos="0 0 1"/>
    <geom name="floor" type="plane" pos="0 0 {z_coord_of_plane}" size="2 2 .1" material="grid"/>
    <site name="anchor" pos="0 0 .3" size=".01"/>
    <camera name="fixed" pos="0 -1.3 .5" xyaxes="1 0 0 0 1 2"/>

    <geom name="pole" type="cylinder" fromto="{pole1_x_pos} 0 -.5 {pole1_x_pos} 0 {z_coord_of_plane + pole1_height}" size=".04"/>
    <body name="bat" pos="{pole1_x_pos} 0 {z_coord_of_plane + pole1_height}">
      <joint name="swing" type="hinge" damping="1" axis="0 0 1"/>
      <geom name="bat" type="capsule" fromto="0 0 .04 0 -.3 .04"
       size=".04" rgba="0 0 1 1"/>
    </body>
    
    <geom name="pole2" type="cylinder" fromto="{pole2_x_pos} 0 -.5 {pole2_x_pos} 0 {z_coord_of_plane + pole2_height}" size=".04"/>
    <body name="bat2" pos="{pole2_x_pos} 0 {z_coord_of_plane + pole2_height}">
      <joint name="swing2" type="hinge" damping="1" axis="0 0 1"/>
      <geom name="bat2" type="capsule" fromto="0 0 .04 0 -.1 .04"
       size=".02" rgba="0 1 1 1"/>
    </body>

    <body name="box_and_sphere" pos="0 0 0">
      <joint name="free" type="free"/>
      <geom name="red_box" type="box" size=".1 .1 .1" rgba="1 0 0 1"/>
      <geom name="green_sphere"  size=".06" pos=".1 .1 .1" rgba="0 1 0 1"/>
      <site name="hook" pos="-.1 -.1 -.1" size=".01"/>
      <site name="IMU"/>
    </body>
  </worldbody>

  <tendon>
    <spatial name="wire" limited="true" range="0 0.35" width="0.003">
      <site site="anchor"/>
      <site site="hook"/>
    </spatial>
  </tendon>

  <actuator>
    <motor name="my_motor" joint="swing" gear="1"/>
    <motor name="my_motor2" joint="swing2" gear="1"/>
  </actuator>

  <sensor>
    <accelerometer name="accelerometer" site="IMU"/>
  </sensor>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(MJCF)
data = mujoco.MjData(model)

In [3]:
n_frames = 180
height = 240
width = 320
frames = []
fps = 16 # 60.0
times = []
sensordata = []

renderer = mujoco.Renderer(model, height, width)

##############################
# constant actuator signal
##############################
mujoco.mj_resetData(model, data)
bat1_power = 5.
bat2_power = 12.
data.ctrl = [bat1_power, bat2_power]

# simulate and render
for i in range(n_frames):
  while data.time < i/fps:
    mujoco.mj_step(model, data)
    times.append(data.time)
    sensordata.append(data.sensor('accelerometer').data.copy())
    
  if i == int(n_frames/2):
    bat2_power =  -2
    data.ctrl = [bat1_power, bat2_power]
    
  renderer.update_scene(data, "fixed")
  frame = renderer.render()
  frames.append(frame.copy())

media.show_video(frames, fps=fps)

This browser does not support the video tag.
